## 1. Configuration

Refer to `README.md` for installation instructions.

Let's start off by sourcing secrets and defining the locations of the various required services:

In [ ]:
# source secrets
source .env

## **2. List TES instances**

Let's see what TES instances we have defined:

In [ ]:
unset TES_INSTANCES
declare -A TES_INSTANCES
while IFS=',' read -r KEY URL; do
    TES_INSTANCES["$KEY"]=$URL
done < .tes_instances

for KEY in "${!TES_INSTANCES[@]}"; do
    echo "$KEY: ${TES_INSTANCES[$KEY]}"
done

## 3. Executing workflows via the TES network

Based off a [previous demonstrator](https://github.com/elixir-cloud-aai/elixir-cloud-demos) showcasing the [cwl-tes](https://github.com/ohsu-comp-bio/cwl-tes), we will demonstrate how a workflow engine with a TES backend can execute workflows on instances of different TES implementations.

We will use the [Snakemake](https://github.com/snakemake/snakemake) workflow engine.

### 3.1 Running Snakemake workflows

As for cwl-tes, we will reuse the "hash splitter" workflow, a simple
workflow with a scatter-gather step. The tasks of the workflow will be executed on individual TESK/Funnnel instances.
Existing files will be overwritten.

In [ ]:
export HOME=/tmp
for KEY in "${!TES_INSTANCES[@]}"; do
    TES="${TES_INSTANCES[$KEY]}"
    echo "Submitting task to $KEY ($TES)..."
    snakemake \
        --directory wf-hashsplitter \
        --workflow wf-hashsplitter/Snakefile \
        --jobs 1 \
        --cores 1 \
        --tes "${TES%/}" \
        --forceall \
        --rerun-incomplete \
        --envvars HOME
    echo "================================================================================"
done
echo "DONE"

### 3.2 Task federation via Snakemake

Now let's point Snakemake to the proTES TES gateway:

In [ ]:
export HOME=/tmp
snakemake \
    --directory wf-hashsplitter \
    --workflow wf-hashsplitter/Snakefile \
    --jobs 1 \
    --cores 1 \
    --tes "${TES_GATEWAY%/}"  \
    --forceall \
    --rerun-incomplete \
    --envvars HOME
echo "================================================================================"
echo "DONE"